## import required lib

In [1]:
!pip install requests beautifulsoup4


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [3]:
URL = "https://www.amazon.in/s?k=laptops&crid=1QRSAH6ZHQWUH&sprefix=laptop%2Caps%2C387&ref=nb_sb_noss_2"

In [4]:
# make header to mimic a browser visit
headers = {
"User-Agent":"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/145.0.0.0 Mobile Safari/537.36"}

In [5]:
#send a get request to the url
response = requests.get(URL, headers= headers)
response

<Response [200]>

In [6]:
#beautifulsoup
soup = BeautifulSoup(response.content,"html.parser")
soup

<!DOCTYPE html>
<html class="a-no-js a-touch a-mobile" data-19ax5a9jf="mongoose" lang="en-in"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta content="width=device-width, maximum-scale=2, minimum-scale=1, initial-scale=1, shrink-to-fit=no" name="viewport"/><meta charset="utf-8"/>
<!-- sp:end-feature:head-start -->
<!-- sp:feature:csm:head-open-part1 -->
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:end-feature:csm:head-open-part1 -->
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link crossorigin="" href="https://images-eu.ssl-images-amazon.com" rel="preconnect"/>
<link crossorigin="" href="https://m.media-amazon.com" rel="preconnect"/>
<!-- sp:end-feature:cs-optimization -->
<!-- sp:feature:csm:head-open-part2 -->
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    

In [7]:
#scrap data from the website

for page in range(1, 3):
    params = {'k' : 'laptops' , 'page' : page}
    response = requests.get(URL, headers= headers, params=params)
    soup = BeautifulSoup(response.content,"html.parser")
    products = soup.find_all("div", {"data-component-type": "s-search-result"})

In [8]:
print(f"Total products found: {len(products)}")


Total products found: 19


In [13]:
import time
data = []
for product in products:
    title_tag = product.find("h2")
    if not title_tag:
        continue
    title_text = title_tag.get_text(strip=True)

#extract the product price
    price_tag = product.find("span", {"class":"a-price-whole"})
    price_text = price_tag.get_text(strip=True) if price_tag else "N/A"

#extract the brand from the title
    brand_match = re.match(r"^(\w+)", title_text)
    brand_text = brand_match.group(1) if brand_match else "N/A"

#extract RAM
    ram_match = re.search(r"(\d+GB RAM)", title_text)
    ram_text = ram_match.group(1) if ram_match else "N/A"

#rating
    rating = product.find("span", {"class": "a-size-mini a-color-base"})
    rating_text = rating.get_text(strip=True) if rating else "N/A"


#Storage ssd and hdd
    storage = re.search(r"(\d+GB SSD|\d+GB HDD)", title_text)
    storage_text = storage.group(1) if storage else "N/A"

#colour 
    colour = re.search(r"(Black|Silver|Grey|Blue|Red)", title_text, re.IGNORECASE)
    colour_text = colour.group(1) if colour else "N/A"

#windows
    windows = re.search(r"(Windows 10|Windows 11)", title_text, re.IGNORECASE)
    windows_text = windows.group(1) if windows else "N/A"







    data.append({
        "Title": title_text,
        "Price": price_text,
        "Brand": brand_text,
        "RAM": ram_text,
        "Rating": rating_text,
        "Storage": storage_text,
        "Colour": colour_text,
        "Windows": windows_text
    })

print(f"Page {page} scraped")
time.sleep(1)

Page 2 scraped


In [14]:
for product in data:
    print("Title: ", product["Title"])
    print("Price: ", product["Price"])
    print("Brand: ", product["Brand"])
    print("RAM: ", product["RAM"])
    print("Rating: ", product["Rating"])
    print("Storage: ", product["Storage"])
    print("Colour: ", product["Colour"])
    print("Windows: ", product["Windows"])
    print("-" * 50)

Title:  Dell 15, Intel Core i5 13th Gen-1334U Processor, 16GB DDR4, 512GB SSD, FHD, 15.6"/39.62cm, Windows 11, MSO 2024, Grey, 1.66kg, [Vostro 3530], 120Hz 250 nits, 15 Month McAfee, Thin & Light, Laptop
Price:  56,990
Brand:  Dell
RAM:  N/A
Rating:  3.7
Storage:  512GB SSD
Colour:  Grey
Windows:  Windows 11
--------------------------------------------------
Title:  ASUS Vivobook 15 (2025), AMD Ryzen 7 7730U Thin & Light Laptop(AMD Radeon iGPU/16GB RAM/512GB SSD/FHD/15.6"/60Hz/Windows 11/M365 Basic (1Year)*/Office Home 2024/Quiet Blue/1.7Kg) M1502YA-BQ989WS
Price:  49,850
Brand:  ASUS
RAM:  16GB RAM
Rating:  4.3
Storage:  512GB SSD
Colour:  Blue
Windows:  Windows 11
--------------------------------------------------
Title:  ASUS Vivobook S16,14th Gen, Intel Core 5-210H,Metallic Design Laptop(Intel UHD iGPU/16GB RAM/512GB SSD/FHD+/16"/144Hz/Windows 11/M365 Basic(1Year)*/Office Home 2024/Silver/1.7Kg) S3607VA-RP114WS
Price:  67,990
Brand:  ASUS
RAM:  16GB RAM
Rating:  4.3
Storage:  512GB